Начало работы

In [ ]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Функция для загрузки звуковых файлов и извлечения MFCC
def load_data_single_folder(folder_path, label):
    X = []
    y = []

    for file in os.listdir(folder_path):
        if file.lower().endswith('.wav'):
            file_path = os.path.join(folder_path, file)
            try:
                # Загрузка аудиофайла
                signal, sr = librosa.load(file_path, sr=None)

                if len(signal) == 0:
                    continue

                # Извлечение MFCC
                mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
                mfccs = np.mean(mfccs.T, axis=0)  # Усреднение

                # Добавление признаков и меток
                X.append(mfccs)
                y.append(label)
            except Exception as e:
                print(f'Ошибка при загрузке файла {file_path}: {e}')

    return np.array(X), np.array(y)

# Путь к папкам с голосами
data_path_sparrow = './sparrow'  # Папка с голосами первого человека
data_path_zyablik = './zyablik'      # Папка с голосами второго человека

# Загружаем данные для каждой папки
X_leha, y_leha = load_data_single_folder(data_path_sparrow, 'sparrow')
X_ja, y_ja = load_data_single_folder(data_path_zyablik, 'zyablik')

# Объединение данных
X = np.concatenate((X_leha, X_ja), axis=0)
y = np.concatenate((y_leha, y_ja), axis=0)

# Преобразование меток
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Сохранение модели
model_filename = 'audio_classification_model.joblib'
joblib.dump(model, model_filename)

# Функция для извлечения MFCC из звуковых файлов
def extract_features(file_paths):
    features_list = []

    for file_path in file_paths:
        signal, sr = librosa.load(file_path, sr=None)

        if len(signal) == 0:
            raise ValueError(f'Ошибка: пустой сигнал из файла {file_path}')

        # Извлечение MFCC
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        mfccs = np.mean(mfccs.T, axis=0)  # Усреднение по времени

        features_list.append(mfccs)

    return np.array(features_list)

# Путь к папке с предсказаниями
pred_voice_path = './pred_voice'

# Автоматически получаем все .wav файлы в указанной папке
pred_file_paths = [os.path.join(pred_voice_path, file_name) for file_name in os.listdir(pred_voice_path) if file_name.lower().endswith('.wav')]

# Загружаем модель
model = joblib.load(model_filename)

# Извлечение признаков
features = extract_features(pred_file_paths)

# Предсказание классов для нескольких файлов
predicted_labels = model.predict(features)
class_names = le.inverse_transform(predicted_labels)

# Вывод результатов
for file_path, class_name in zip(pred_file_paths, class_names):
    print(f'Файл {os.path.basename(file_path)} классифицирован как: {class_name}')

FileNotFoundError: [Errno 2] No such file or directory: './sparrow'

In [ ]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

# Функция для загрузки звуковых файлов и извлечения MFCC
def load_data_from_multiple_folders(base_folder_path):
    X = []
    y = []

    for label in os.listdir(base_folder_path):
        label_folder_path = os.path.join(base_folder_path, label)
        if os.path.isdir(label_folder_path):
            for file in os.listdir(label_folder_path):
                if file.lower().endswith('.wav'):
                    file_path = os.path.join(label_folder_path, file)
                    try:
                        # Загрузка аудиофайла
                        signal, sr = librosa.load(file_path, sr=None)

                        if len(signal) == 0:
                            continue

                        # Извлечение MFCC
                        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
                        mfccs = np.mean(mfccs.T, axis=0)  # Усреднение

                        # Добавление признаков и меток
                        X.append(mfccs)
                        y.append(label)
                    except Exception as e:
                        print(f'Ошибка при загрузке файла {file_path}: {e}')

    return np.array(X), np.array(y)

# Путь к папке на Google Диске
data_path = '/content/drive/My Drive/sorted_bird_sounds'  # Укажите путь к папке с голосами птиц

# Загружаем данные для всех папок с птицами
X, y = load_data_from_multiple_folders(data_path)

# Преобразование меток
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)

# Получение уникальных меток, присутствующих в тестовой выборке
unique_labels = np.unique(y_test)

# Проверка на совпадение классов
try:
    report = classification_report(y_test, y_pred, target_names=le.inverse_transform(unique_labels))
    print(report)
except ValueError as e:
    # Если возникает ошибка, просто продолжим с имеющимися метками
    print(f"Предупреждение: {e}")
    print("Печатаем отчет с доступными метками:")
    print(classification_report(y_test, y_pred))

# Сохранение модели
model_filename = 'audio_classification_model.joblib'
joblib.dump(model, model_filename)

# Функция для извлечения MFCC из звуковых файлов
def extract_features(file_paths):
    features_list = []

    for file_path in file_paths:
        signal, sr = librosa.load(file_path, sr=None)

        if len(signal) == 0:
            raise ValueError(f'Ошибка: пустой сигнал из файла {file_path}')

        # Извлечение MFCC
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        mfccs = np.mean(mfccs.T, axis=0)  # Усреднение по времени

        features_list.append(mfccs)

    return np.array(features_list)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Предупреждение: Number of classes, 419, does not match size of target_names, 415. Try specifying the labels parameter
Печатаем отчет с доступными метками:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         2
           2       0.00      0.00      0.00         1
           3       1.00      0.67      0.80         3
           4       0.67      1.00      0.80         2
           5       1.00      0.67      0.80         3
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       0.67      0.67      0.67         3
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         2
          12 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_